In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from toolkit import slice_spectrum, concatenate_spectra, bands_TiO, match_spectra, SimpleSpectrum
import astropy.units as u
from scipy.optimize import fmin_l_bfgs_b

In [2]:
def plot_spliced_spectrum(observed_spectrum, model_flux, other_model=None):
    n_chunks = len(observed_spectrum.wavelength_splits)
    fig, ax = plt.subplots(n_chunks, 1, figsize=(8, 10))

    for i, inds in enumerate(observed_spectrum.wavelength_splits):
        min_ind, max_ind = inds
        
        ax[i].errorbar(observed_spectrum.wavelength[min_ind:max_ind].value, 
                       observed_spectrum.flux[min_ind:max_ind], 
                       0.025*np.ones(max_ind-min_ind))
        ax[i].plot(observed_spectrum.wavelength[min_ind:max_ind], 
                   model_flux[min_ind:max_ind])
        
        if other_model is not None:
            ax[i].plot(observed_spectrum.wavelength[min_ind:max_ind], 
                       other_model[min_ind:max_ind], alpha=0.4)
        
        ax[i].set_xlim([observed_spectrum.wavelength[min_ind].value,
                        observed_spectrum.wavelength[max_ind-1].value])
        ax[i].set_ylim([0.9*observed_spectrum.flux[min_ind:max_ind].min(), 
                        1.1])

    return fig, ax

In [7]:
import h5py

archive = h5py.File('/Users/bmmorris/git/aesop/notebooks/spectra.hdf5', 'r+')

In [9]:
list(archive['HD110833'])

['2017-03-17T05:47:24.899', '2017-03-17T05:54:59.760']

In [11]:
targets = list(archive)
spectrum1 = archive['HATP11']['2017-06-12T07:28:06.310'] # K4
wavelength1 = spectrum1['wavelength'][:]
flux1 = spectrum1['flux'][:]
target = SimpleSpectrum(wavelength1, flux1, dispersion_unit=u.Angstrom)

spec_band = []

first_n_bands = 5
width = 3

for band in bands_TiO[:first_n_bands]:
    target_slice = slice_spectrum(target, band.min-width*u.Angstrom, band.max+width*u.Angstrom)
    target_slice.flux /= target_slice.flux.max()
    spec_band.append(target_slice)

target_slices = concatenate_spectra(spec_band)

spectrum2 = archive['HD110833']['2017-03-17T05:47:24.899']#archive[target][time]
wavelength2 = spectrum2['wavelength'][:]
flux2 = spectrum2['flux'][:]
source1 = SimpleSpectrum(wavelength2, flux2, dispersion_unit=u.Angstrom)

spec_band = []
for band, inds in zip(bands_TiO[:first_n_bands], target_slices.wavelength_splits):
    target_slice = slice_spectrum(source1, band.min-width*u.Angstrom, band.max+width*u.Angstrom, 
                                  force_length=abs(np.diff(inds))[0])
    target_slice.flux /= target_slice.flux.max()
    spec_band.append(target_slice)

source1_slices = concatenate_spectra(spec_band)

In [12]:
def chi2(p, target, comp1, comp2):
    model, residuals = instr_model(target, comp1, comp2, *p)
    return residuals

In [14]:
from toolkit import instr_model
from astropy.utils.console import ProgressBar
bfgs_options_fast = dict(epsilon=1e-3, approx_grad=True,
                         m=10, maxls=20)
bfgs_options_precise = dict(epsilon=1e-3, approx_grad=True,
                            m=30, maxls=50)

target_list = []
time_list = []
residuals_list = []
area_list = []
counter = 0
for target in archive: 
    for time in archive[target]:
        counter +=1 

for target in archive: 
    for time in archive[target]:
        if target != 'HATP11' and target != 'HD110833':

            spectrum3 = archive[target][time]
            wavelength3 = spectrum3['wavelength'][:]
            flux3 = spectrum3['flux'][:]
            source2 = SimpleSpectrum(wavelength3, flux3, dispersion_unit=u.Angstrom)

            spec_band = []
            for band, inds in zip(bands_TiO[:first_n_bands], target_slices.wavelength_splits):
                target_slice = slice_spectrum(source2, band.min-width*u.Angstrom, band.max+width*u.Angstrom, 
                                              force_length=abs(np.diff(inds))[0])
                target_slice.flux /= target_slice.flux.max()
                spec_band.append(target_slice)

            source2_slices = concatenate_spectra(spec_band)

            bounds = [[-30, 0], [0, 15]] + first_n_bands*[[-2, 2]]
            initp = [-0.5, 1] + first_n_bands*[0.0]

            result = fmin_l_bfgs_b(chi2, initp, bounds=bounds, 
                                   args=(target_slices, source1_slices, source2_slices),
                                   **bfgs_options_precise)

            model, resid = instr_model(target_slices, source1_slices, source2_slices, *result[0])

            target_list.append(target)
            time_list.append(time)
            residuals_list.append(resid)
            area_list.append(np.exp(result[0][0]))
            print(target, resid, np.exp(result[0][0]))

/Users/bmmorris/anaconda/lib/python3.5/site-packages/scipy/signal/windows.py:1217: RuntimeWarning: divide by zero encountered in true_divide
  w = np.exp(-n ** 2 / sig2)


51Peg 0.42806957353 6.09214322861e-06
51Peg 0.428078807984 1.96138066803e-05
51Peg 0.428072923665 1.53755209764e-06
51Peg 0.428073951612 6.15176791769e-07
GJ4099 0.618733355549 5.77051345254e-05
GJ4099 0.618720496293 2.74433351389e-05
GJ702B 0.576631718331 0.210954989501
GJ9781A 0.572638098543 0.257955595409
HD10697 0.593622437945 1.04349002481e-06
HD113827 0.597915763633 0.0209035290242
HD122120 0.40864549387 1.0
HD127506 0.428098252953 0.000266886523805
HD129333 0.428074013646 4.83805932232e-07
HD134319 0.607604136269 7.58745973016e-06
HD14039 0.607604298033 1.71321492459e-05
HD14039 0.59806811558 7.21880356334e-07
HD148467 0.428074563009 2.56852699546e-05
HD149957 0.587426504339 0.168839380539
HD151288 0.590949372554 0.0619337749524
HD175742 0.428067713178 3.69214278677e-06
HD178126 0.428710996301 0.000482217800509
HD182488 0.428078074116 3.28444428633e-05
HD182488 0.428081763488 9.24040711543e-05
HD182488 0.42808033602 1.67874711352e-05
HD182488 0.428079098458 2.04128781565e-05
HD2

In [15]:
first_percentile = np.percentile(residuals_list, 2)

In [16]:
for target, time, resid, area in zip(target_list, time_list, residuals_list, area_list):
    if first_percentile > resid: 
        print(target, time, resid, area)

HD122120 2017-06-15T03:52:13.690 0.40864549387 1.0
HR8832 2017-09-05T04:40:59.710 0.427637232699 0.0111955887476
